# Building the Simple Calculator Skill
This notebook demonstrates how to use the Cortex Python SDK to build a simple Skill.  For simple cases, the entire Skill can be defined, tested, and deployed right from a notebook.

In [ ]:
# Common Setup
%run setup.ipynb
cortex = Cortex.client()
builder = cortex.builder()

## Define the simple_calc function


In [ ]:
%%cortex_action --name 'class/simple-calc-<your_initials>' --function simple_calc
from cortex import Message

def simple_calc(params):
    msg = Message(params)
    eq = msg.payload.get('eq')
    soln = eval(eq)
    return Message.with_payload({'num': int(soln)}).to_params()

Build and Deploy Action

### Testing Actions
Using the Cortex client, we can test our Action to make sure it deployed properly.

In [ ]:
# Instantiate Cortex Client
cortex = Cortex.client()

# Retrieve our Action that was deployed above
action = cortex.action('class/simple-calc-<your_initials>')

The Action deployment status should say **COMPLETED**.  This will indicate that our Action is ready to invoke.

In [ ]:
action.get_deployment_status()

Invoke the Action using a Message object.  Here we just pass in the expected _eq_ parameter in the Message payload.

In [ ]:
from cortex import Message
rs = action.invoke(Message.with_payload({'eq': '5 + 5'}))
rs.payload

## Building a Cortex Skill
Now that our Action is ready and tested, we can move on to building a Cortex Skill.  In this simple example, our Skill will just pipe an Input to our Action and route the Output back to the caller.

In [ ]:
builder = cortex.builder()

The _builder_ has multiple entry points, we use the _skill_ method here to declare a new "Simple Calculator" Skill.  Each _builder_ method returns an instance of the builder so we can chain calls together.  

In [ ]:
b = builder.skill('class/simple-calc-<your_initials>').title('Simple Calculator-<your_initials>').description('Skill which solves a given equation by <your_name>')

Next, we use the Input sub-builder to construct our Skill Input.  This is where we declare how our Input will route messages.  In this simple case, we use the _all_ routing which routes all input messages to same Action for processing and declares wich Output to route Action outputs to.  We pass in our Action from the previous section to wire the Skill to the Action (we could have also passed in the Action name here).  Calling _build_ on the Input will create the input object, add it to the Skill builder, and return the Skill builder.

In [ ]:
b = b.input('equation').title('Equation in').parameter(name='eq', type='string').all_routing(action, 'result').build()


In the previous step, we referenced an Output called **result**.  We can create that Output here using the Output sub-builder.

In [ ]:
b = b.output('result').title('Equation Out').parameter(name='num', type='number').build()

## Preview the CAMEL Document
We can preview the CAMEL document that each builder will create using the _to_camel_ method.

In [ ]:
b.to_camel()

## Final Build and Publish
We are now ready to build and publish our Skill.

In [ ]:
skill = b.build()
print('%s (%s) v%d' % (skill.title, skill.name, skill.version))

In [ ]:
rs = skill.invoke(input_name='equation', message=Message.with_payload({'eq': '12500 * 5'}))
rs.payload